In [ ]:
#installing the needed packages

!pip install openpyxl
!pip install contractions

In [ ]:
#reading your file into the dataframe

import pandas as pd

df = pd.read_excel('../input/example2/Example1.xlsx')
#df = pd.read_excel('../input/example1/Example1.xlsx') <- this is my file path. Yours should be sth similar

In [ ]:
#take a look at your dataframe

df

We're -> Were 

In [ ]:
#We define the functions for text preprocessing
import contractions
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

print('Functions have been successfully defined!')

In [ ]:
new_col = []
for item in df['Text']:
    new_abs = contractions.fix(item)
    new_abs = remove_special_characters(new_abs, remove_digits=True)
    new_abs_words = new_abs.split()
    new_abs_words = [lemmatizer.lemmatize(w).lower() for w in new_abs_words if w.lower() not in stop]
    new_col.append(new_abs_words)
df['Words'] = new_col

In [ ]:
df

In [ ]:
#finding all unique words
all_words = []
for item in df['Words']:
    new_abs_words =item
    all_words += new_abs_words
all_words_unique = list(set(all_words))

print('There are ' + str(len(all_words_unique)) + ' unique words in our dataset.')

In [ ]:
#making document term matrix
word_matrix = {}
for word in all_words_unique:
    word_vec = []
    for item in df['Words']:
        if word in item:
            word_vec += [1]
        else:
            word_vec += [0]
    word_matrix[word] = word_vec

dc_df = pd.DataFrame(word_matrix)
dc_df

In [ ]:
#Calculating Cosine Similarity

from sklearn.metrics.pairwise import cosine_similarity

cos_matrix = cosine_similarity(dc_df)

pd.DataFrame(cos_matrix)

In [ ]:
#Calculating Euclidean Distance

from sklearn.metrics.pairwise import euclidean_distances

euc_matrix = euclidean_distances(dc_df)

pd.DataFrame(euc_matrix)

In [ ]:
#Checking the most similar document 

#specify the index of your chosen document
chosen_doc = 4
scores = sorted(cos_matrix[chosen_doc],reverse=True) 
score = scores[1]                                     
result_doc = list(cos_matrix[chosen_doc]).index(score)
# note: you may want to change cos_matrix to euc_matrix and set reverse=False when you are using 
#       Euclidean distance since the smaller the Euclidean distance is, the similar the two documents are.
       
print('The document that is the most similar with document ' + str(chosen_doc) + ' is ' + 'document ' + str(result_doc) + '.')

In [ ]:
#tf -- Term Frequency
tf_matrix = {}
tf_ranking ={}
for word in all_words_unique:
    word_vec = []
    for item in df['Words']:
        if word in item:
            word_vec += [item.count(word)/len(item)]
        else:
            word_vec += [0]
    tf_matrix[word] = word_vec

pd.DataFrame(tf_matrix)

In [ ]:
#Top frequenty terms

def top_terms(matrix_df, n=10): #input should be a pandas dataframe
    output_dict = {}
    for index, series in matrix_df.iterrows():
        doc_num = 'doc' + str(index)
        scores = dict(series)
        scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
        terms = scores_sorted.keys()
        terms_topn = list(terms)[:n]
        output_dict[doc_num] = terms_topn
    output_df = pd.DataFrame(output_dict)
    return output_df.transpose()

matrix_df = pd.DataFrame(tf_matrix)
tf_ranking = top_terms(matrix_df, 10)
tf_ranking

In [ ]:
import math

#idf -- Inverse document frequency
idf_matrix = {}
for word in word_matrix:
    idf_matrix[word] = math.log(len(df['Words'])/sum(word_matrix[word]))

print("Inverse Document Frequency Matrix successfully computed!")

In [ ]:
#Ranking of the Inverse document frequency

idf_ranking = {k: v for k, v in sorted(idf_matrix.items(), key=lambda item: item[1], reverse=True)}
idf_ranking

In [ ]:
#Computing tfidf

tfidf = {}
for word in idf_matrix:
    idf = idf_matrix[word]
    tfidf_vec = tf_matrix[word]
    tfidf[word] = [i * idf for i in tfidf_vec]

pd.DataFrame(tfidf)

In [ ]:
#Ranking according to tfidf

tfidf_df = pd.DataFrame(tfidf)
top_tfidf_terms = top_terms(tfidf_df)

top_tfidf_terms

In [ ]:
#Calculating Cosine Similarity using tfidf

dc_df = pd.DataFrame(tfidf)
cos_matrix = cosine_similarity(dc_df)

pd.DataFrame(cos_matrix)

In [ ]:
#Checking the similarity between two documents using tfidf

chosen_doc = 2
scores = sorted(cos_matrix[chosen_doc],reverse=True)
score = scores[1]
result_doc = list(cos_matrix[chosen_doc]).index(score)

print('The document that is the most similar with document ' + str(chosen_doc) + ' is ' + 'document ' + str(result_doc) + '.')

In [ ]:
#Take a look at your text

print(df.Text[4])
print(df.Text[3])

In [ ]:
#Your may also locate your data using the title of it
df.loc[df['Title']=='Cancer Progress and Priorities: Lung Cancer', ['Text']]